In [35]:
source("00_functions.R")
require(ggplot2)


 loading required packages



In [44]:
standardise_time = function(df, axis, census_time){
    
    # select the axis that I want
    df = df[, axis]

    plts = unique(unlist(strsplit(rownames(df), "_"))[c(T, F, F)])

    if (exists('out')){ rm(out) }
        
    for (plt in plts){
    
        tmp1 = df[which(unlist(strsplit(rownames(df), "_"))[c(T, F, F)] == plt), ]

        splt = unique(unlist(strsplit(rownames(tmp1), "_"))[c(F, T, F)])
        cens = sort(unique(unlist(strsplit(rownames(tmp1), "_"))[c(F, F, T)]))

        raw = array(dim= c(length(splt), ncol(df), length(cens)),
              dimnames = list(splt, axis, cens))

        adj = raw

        for (sp in splt){ for (cen in cens){
            raw[sp, , cen] = unlist(tmp1[paste(plt, sp, cen, sep = "_"), ])
        }}

        adj[,,cens[1]] = raw[,,cens[1]]

        for (c in 2:length(cens)){

            mask = census_time$plot == plt & census_time$census == cens[c]

        if (sum(mask) == 0) {
            adj[ , , cens[c]] = NA
        } else {
            diff_yrs = census_time[mask, "diff_yrs"]
            adj[,,cens[c]] = raw[,,cens[c-1]] + ((raw[,,cens[c]] - raw[,,cens[c-1]])*(1/diff_yrs)) 
        }}

        adj = adply(adj, c(1, 3))

        colnames(adj)[1:2] = c('subplot', 'census')
        adj$plot      = plt
        rownames(adj) = paste(adj$plot, adj$subplot, adj$census, sep = "_")

        adj = adj[order(rownames(adj)), ]

        adj = na.omit(adj)

        if (exists('out')){
            out = rbind(out, adj)
        } else {
            out = adj
        }
    }
    return(out)
}

In [45]:
mamls_df = read.csv("../Results/m_trap-year.csv", row.names=1)

mamls_cen = read.csv("../Results/mammals_years_dates.csv")

pca.rslt_mamls  = do_pca(mamls_df, scale = F, plot = F)

cat("Explained Varience Mammals:")
pca.rslt_mamls@exp.var[0:3]

mamls = hvs_rslts(pca.rslt_mamls@axis, axis = c("PC1", "PC2", "PC3"), what = "seq", census_time = mamls_cen)

Explained Varience Mammals:

PC1       PC2       PC3 
0.2644064 0.2403138 0.1133896


Building Hypervolume  28  of  28 :  O_2013
Comparing Hypervolume 24  of  24 :  O_2016-2017


In [47]:
trees_df = read.csv("../Results/trees_matrix.csv", row.names=1)

trees_cen = read.csv("../Results/trees_census_dates.csv")

pca.rslt_trees  = do_pca(trees_df, scale = F, plot = F)

cat("Explained Varience Mammals:")
pca.rslt_trees@exp.var[0:3]

trees = hvs_rslts(pca.rslt_trees@axis, axis = c("PC1", "PC2", "PC3"), what = "seq", census_time = trees_cen)

Explained Varience Mammals:

PC1        PC2        PC3 
0.28892554 0.05653079 0.04438155


Building Hypervolume  36  of  36 :  Tower_c4 
Comparing Hypervolume 27  of  27 :  Tower_c3-c4 


In [ ]:
plot_hvs(trees)